# Web Scraping I.


Web-Scrapping es la forma que tenemos para referirnos a la captura de información de cualquier sitio web. Su objetivo es capturar información de forma automática.

La librería principal que vamos a utilizar es beautifulsoup. Para ello, hay que instalarla primero:
En el símbolo del sistema:

>pip install beautifulsoup4


In [ ]:
!pip install beautifulsoup4

También vamos a instalar request:

>pip install requests

Una vez instalado, se importa en el código:

In [ ]:
from bs4 import BeautifulSoup
import requests

El proceso se divide principalmente en tres fases:

- Carga de la dirección web a la que realizar el scrapping. A través de  requests.

- Extracción del contenido de la web a partir de Beautifulsoup

- Manipulación del contenido.


## 1. Carga de la URL

Request necesita dos cosas: La URL y un elemento cabecera. El elemento cabecera es vital, ya que muchas web no permiten el acceso a la información a no ser que detecten que las peticiones vienen de un navegador. Por ejemplo Mozilla Firefox.

Así, definimos headers como un diccionario cuyo User-Agent es Firefox:

In [ ]:
headers = {'User-Agent': 'Firefox'}
# la url puede ser de cualquier página y será el objetivo de nuestro web-scrapper
url = 'https://resultados.as.com/resultados/futbol/primera/2015_2016/jornada/regular_a_1/'

In [ ]:
re = requests.get(url,headers=headers)

Ahora debemos preguntar si la conexion con esa pagina web ha funcionado. Es decir si la propiedad status_code que devuelve re es 200 o 201. Si es así, pasamos a la siguiente fase.

## 2. Extracción del contenido con BeautifulSoup

Para ello, pasamos el contenido que ha obtenido re y que se encuentra en re.text a través de un parser de HTML en caso de que sea HTML u otro, en caso de que sea XML, etc.

In [ ]:
print(re.status_code)

In [ ]:
if re.status_code == 200:
    soup = BeautifulSoup( re.text, 'html.parser')
    # print(soup.prettify())
    #  y ahora preguntamos si se ha cargado algo de texto
    if soup is not None:
        # en articles vamos colocando todas las que cumplan tener la etiqueta li, y la clase
        articles = soup.find_all('li',{'class' : 'list-resultado'})
        for article in articles:
            # y ahora iteraríamos para sacar todo lo necesario
            # estadio
            lugares = article.find_all('span',{'itemprop':'location'})
            equipo = article.find('span',{'class' : 'nombre-equipo'}).getText()
            enlaces = article.find_all('div', {'class':'cont-resultado finalizado'})
            for enlace in enlaces:
                direccion = enlace.find('a').get('href')
                print(direccion)

In [ ]:
# podríamos ver el contenido de lo que ha capturado con esta instrucción:
print(soup.prettify())

In [ ]:
# o también mostrar la lista de etiquetas como una lista con el metodo children.
list(soup.children)

In [ ]:
# Podemos acceder a cada elemento con .(Nombre del tag)
soup.head

In [ ]:
soup.head.title

In [ ]:
# Con .string accedemos al contenido
soup.head.title.string

## 3. Manipulación del contenido

En esta fase vamos iterando sobre lo que hemos conseguido con BeautifulSoup y que lo vamos almacenando en objetos como articles. 

Si el contenido es directamente la información que se encuentra en lo encontrado por soup.find.all, sólo hay que imprimir o guardar.

Sin embargo a veces hay que volver a llamar a otras búsquedas find, etc.:

In [ ]:
# fecha y hora
masinfo= article.find_all('div',{'class' : 'info-evento'})
for info in masinfo:
    fechahora = info.find('time').get('content')
    # una vez aquí ya tenemos contenido real
    print(fechahora)

En el anterior código, vemos que para obtener la fecha y la hora, simplemente utilizamos find. Este es el caso cuando no hay una lista de elementos. Suele devolver un dato ya disponible, no un objeto iterable.

Otra alternativa que podemos probar cuando este get no funcione porque necesitemos especificar la clase es:

    resultado = enlace.find('a',{'class' : 'resultado'}).getText()

## Conclusión

El proceso de Web-Scrapping no es un proceso complicado, pero si tedioso.
Y es tedioso porque hay que comprender cuál es la estructura de la web que queremos scrappear y es posible que con el tiempo, un web-scrapper que funcionaba, no nos funciona actualmente por haber cambiado la estructura de la web.

## Ejercicios

**8.2.1** De la página de bolsa de madrid: https://www.bolsamadrid.es/esp/aspx/Indices/Resumen.aspx obten un daframe con los contenidos de la tabla.